# Analysis of *A. thaliana* RNA-Seq data with pyrpipe 
In this case study, we will utilize *A. thaliana* public RNA-Seq data and perform transcript assembly.

In [26]:
from pyrpipe import sra,mapping,assembly,qc,tools
from pyrpipe import pyrpipe_utils as pu
from pyrpipe import pyrpipe_engine as pe
#First get the srr accessions of the runs. For this one can use the python package pysradb or R package sradb
#i will consider following randomly selected accessions
#athalRuns=['SRR976159','SRR978411','SRR978410','SRR971778','SRR1058116','SRR1058118','SRR1058121','SRR1058110','SRR1058120','SRR1058117','SRR1104134','SRR1104133','SRR1104135','SRR1104136','SRR1105825']
athalRunsSmol=['SRR976159','SRR978411','SRR971778']
#set your working directory if you don't want to use the current working directory
workingDir="athal_out"
#create working directory
if not pu.check_paths_exist(workingDir):
    pu.mkdir(workingDir)

## Download genome and gtf

In [28]:
GENOME=workingDir+"/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa"
GTF=workingDir+"/Arabidopsis_thaliana.TAIR10.45.gtf"

if not pu.check_files_exist(GENOME):
    print("Downloading genome fasta file")
    wget="wget ftp://ftp.ensemblgenomes.org/pub/release-46/plants/fasta/arabidopsis_thaliana/dna/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa.gz -q -O "+GENOME+".gz"
    pe.execute_command(wget.split(),verbose=False,logs=False)
    pe.execute_command(['gunzip',GENOME+".gz"],verbose=True,logs=False)
else:
    print('File {} exists'.format(GENOME))
    
if not pu.check_files_exist(GTF):
    print("Downloading GTF file")
    wget="wget ftp://ftp.ensemblgenomes.org/pub/release-46/plants/gtf/arabidopsis_thaliana/Arabidopsis_thaliana.TAIR10.46.gtf.gz -O "+GTF+".gz"
    pe.execute_command(wget.split(),verbose=False,logs=False)
    pe.execute_command(['gunzip',GTF+".gz"],verbose=True,logs=False)
else:
    print('File {} exists'.format(GTF))
    
        

File athal_out/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa exists
File athal_out/Arabidopsis_thaliana.TAIR10.45.gtf exists


## Download data and create SRA objects

Using the pyrpipe sra module, we will create `SRA` objects for each of the run accession. pyrpipe's SRA class will automatically fetch the fastq files upon creation an object.

In [29]:

##download all data in athalRuns
sraObjects=[]

for x in athalRunsSmol:
    try:
        thisSraOb=sra.SRA(x,workingDir)
        sraObjects.append(thisSraOb)
    except:
        print('Failed to download {}'.format(x))
    

print("Following runs downloaded:")
for ob in sraObjects:
    print(ob.srr_accession)

Start:21-01-01 14:34:18
$ prefetch -O athal_out/SRR976159 SRR976159
End:21-01-01 14:34:26
Time taken:0:00:08
Start:21-01-01 14:34:26
$ fasterq-dump -O athal_out/SRR976159 -o SRR976159.fastq -e 6 -f athal_out/SRR976159/SRR976159.sra
End:21-01-01 14:34:51
Time taken:0:00:25
Start:21-01-01 14:34:51
$ prefetch -O athal_out/SRR978411 SRR978411
End:21-01-01 14:35:00
Time taken:0:00:09
Start:21-01-01 14:35:00
$ fasterq-dump -O athal_out/SRR978411 -o SRR978411.fastq -e 6 -f athal_out/SRR978411/SRR978411.sra
End:21-01-01 14:35:22
Time taken:0:00:22
Start:21-01-01 14:35:22
$ prefetch -O athal_out/SRR971778 SRR971778
End:21-01-01 14:35:34
Time taken:0:00:11
Start:21-01-01 14:35:34
$ fasterq-dump -O athal_out/SRR971778 -o SRR971778.fastq -e 6 -f athal_out/SRR971778/SRR971778.sra


Following runs downloaded:
SRR976159
SRR978411
SRR971778


End:21-01-01 14:36:20
Time taken:0:00:46


## Saving current session

This is a good place to demonstrate saving the pyrpipe session. 
**In a typical HPC setting, one might have access to special data-transfer nodes**. 
These nodes could be used for downloading data efficiently but does not allow expensive computations. 
On the other hand data may be downloaded on compute nodes **but that will burn most of the computing time/allocations for only downloading the data**. 
Thus it might be a good idea to download data separately and then start the processing.
We can save the objects created with pyrpipe and restore our session later on a compute node.


In [19]:
# save current session
from pyrpipe import pyrpipe_session
pyrpipe_session.save_session(filename="mySession",add_timestamp=True,out_dir=workingDir)

Session saved to: athal_out/mySession_20210101142026.pyrpipe


True

## Restoring saved session
We can restore the pyrpipe session using the saved session file (saved with .pyrpipe extension).

**Note** After restoring session a new log file will generated to store the logs.

In [20]:
#first clear current session used by notebook
%reset
print(sraObjects)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


NameError: name 'sraObjects' is not defined

In [21]:
#restore session
from pyrpipe import pyrpipe_session
#copy and paste the pyrpipe session file below
st=pyrpipe_session.restore_session("athal_out/mySession_20210101142026.pyrpipe")
print(sraObjects)

Session restored.
[<pyrpipe.sra.SRA object at 0x7f7c8c9b1310>, <pyrpipe.sra.SRA object at 0x7f7c8c9c8410>, <pyrpipe.sra.SRA object at 0x7f7c8c9c8b10>]


## Performing fastq quality control

Each `SRA` object stores the path to the fastq files. We can directly use the `SRA` object's `trim()` function to perform trimming of fastq-files. To do this we need to specify a `RNASeqQC` object, from the `qc` module.
At present, pyrpipe implements two classes `Trimgalore` and `BBmap` in the qc module.
We can create an object of either `Trimgalore` or `BBmap` and pass it to the `trim()` function.

In [30]:
#using bbduk
pathToAdapters="adapters2.fa"
#arguments to pass to bbduk; note these can go in ./params/bbduk.yaml
bbdOpts={"ktrim":"r","k":"23","mink":"11","qtrim":"'rl'","trimq":"10","ref":pathToAdapters}
#an object for running bbduk.sh with specified parameters
bbdOb=qc.BBmap(threads=4,memory=2,**bbdOpts)

#start QC
for ob in sraObjects:
    ob.trim(bbdOb,delete_original=False) #delete_original will delete the untrimmed fastq
    
#after finishing view the current fastq files in the sra objects

for ob in sraObjects:
    print("SRR Accession: {}, fastq files: {}. {}".format(ob.srr_accession,ob.fastq_path,ob.fastq2_path))
    
    if ob.fastq_exists():
          print("Both files exist!!")
    else:
          print("Error")
          raise Exception("Fastq files not found")

Start:21-01-01 14:37:00
$ bbduk.sh ktrim=r k=23 mink=11 qtrim='rl' trimq=10 ref=adapters2.fa threads=4 in=athal_out/SRR976159/SRR976159_1.fastq in2=athal_out/SRR976159/SRR976159_2.fastq out=athal_out/SRR976159/SRR976159_1_bbduk.fastq out2=athal_out/SRR976159/SRR976159_2_bbduk.fastq -Xmx2g
End:21-01-01 14:37:51
Time taken:0:00:51
Start:21-01-01 14:37:51
$ bbduk.sh ktrim=r k=23 mink=11 qtrim='rl' trimq=10 ref=adapters2.fa threads=4 in=athal_out/SRR978411/SRR978411_1.fastq in2=athal_out/SRR978411/SRR978411_2.fastq out=athal_out/SRR978411/SRR978411_1_bbduk.fastq out2=athal_out/SRR978411/SRR978411_2_bbduk.fastq -Xmx2g
End:21-01-01 14:38:38
Time taken:0:00:47
Start:21-01-01 14:38:38
$ bbduk.sh ktrim=r k=23 mink=11 qtrim='rl' trimq=10 ref=adapters2.fa threads=4 in=athal_out/SRR971778/SRR971778_1.fastq in2=athal_out/SRR971778/SRR971778_2.fastq out=athal_out/SRR971778/SRR971778_1_bbduk.fastq out2=athal_out/SRR971778/SRR971778_2_bbduk.fastq -Xmx2g


SRR Accession: SRR976159, fastq files: athal_out/SRR976159/SRR976159_1_bbduk.fastq. athal_out/SRR976159/SRR976159_2_bbduk.fastq
Both files exist!!
SRR Accession: SRR978411, fastq files: athal_out/SRR978411/SRR978411_1_bbduk.fastq. athal_out/SRR978411/SRR978411_2_bbduk.fastq
Both files exist!!
SRR Accession: SRR971778, fastq files: athal_out/SRR971778/SRR971778_1_bbduk.fastq. athal_out/SRR971778/SRR971778_2_bbduk.fastq
Both files exist!!


End:21-01-01 14:39:41
Time taken:0:01:04


## Aligning clean reads to the reference genome

After finishing fastq quality control we will map reads to the reference genome. To do this, first we need to have an `Aligner` object. The pyrpipe `mapping` module defines three classes `Star`, `Hisat2` and `Bowtie2`. These classes provide an API to STAR, Hisat2 and Bowtie2 alignment tools. We can diretcly used the `SRA` object's `align()` function to generate the bam files. 

After aligning and generating bam files, the bam file path will be stored with the `SRA` object in the `bam_path` attribute.

In this example we will use Hisat2 to align the reads. First we define a Hisat2 object and provide idex as 'athalIndex/athalInd'. This index doesn't exist. To create one index we also provide a genome file as argument. The following statement will create a Hisat2 object and generate an index using default parameters.

**Note: It is recommended that users generate their index using appropriate parameters. Parameters to be used while building an index could be stored in hisat2_index.yaml or star_index.yaml files and pyrpipe will automatically load them if building a new index.**

In [33]:
#using hisat2
hsOpts={"--dta-cufflinks":"","-p":"6"}
hs=mapping.Hisat2(index=workingDir+"/athalIndex/athalInd",genome=GENOME,threads=6,**hsOpts)

Start:21-01-01 14:53:58
$ hisat2-build -p 6 athal_out/Arabidopsis_thaliana.TAIR10.dna.toplevel.fa athal_out/athalIndex/athalInd
End:21-01-01 14:54:34
Time taken:0:00:35


After the index is generated, it is stored with the Hisat2 object, as `index` attribute. We can now use the hisat object to run the alignment step.
**Note: pyrpipe will automatically convert the SAM files output by Hisat2 step to sorted BAM files using Samtools.**

In [34]:
#start alignment
bamList=[]
for ob in sraObjects:
    print("Processing {}...".format(ob.srr_accession))
    ob.align(hs)
    bamList.append(ob.bam_path)
    
    ##Other way to perform alignment is to use hs.perform_alignment and pass SRA object to it
    #thisBam=hs.perform_alignment(ob,**hsOpts) #note the parameter p supplied here will replace the parameter "threads" passed during object construction
    #if thisSam:
    #    bamList.append(thisBam)
print("Alignment done!! Bam files:"+ ",".join(bamList))    

Processing SRR976159...


Start:21-01-01 14:55:15
$ hisat2 --dta-cufflinks -p 6 -x athal_out/athalIndex/athalInd -1 athal_out/SRR976159/SRR976159_1_bbduk.fastq -2 athal_out/SRR976159/SRR976159_2_bbduk.fastq -S athal_out/SRR976159/SRR976159_hisat2.sam
End:21-01-01 14:56:20
Time taken:0:01:05
Start:21-01-01 14:56:20
$ samtools view -@ 6 -o athal_out/SRR976159/SRR976159_hisat2.bam -b athal_out/SRR976159/SRR976159_hisat2.sam
End:21-01-01 14:56:55
Time taken:0:00:34
Start:21-01-01 14:56:55
$ samtools sort -@ 6 -o athal_out/SRR976159/SRR976159_hisat2_sorted.bam athal_out/SRR976159/SRR976159_hisat2.bam
End:21-01-01 14:57:17
Time taken:0:00:22
Start:21-01-01 14:57:17
$ hisat2 --dta-cufflinks -p 6 -x athal_out/athalIndex/athalInd -1 athal_out/SRR978411/SRR978411_1_bbduk.fastq -2 athal_out/SRR978411/SRR978411_2_bbduk.fastq -S athal_out/SRR978411/SRR978411_hisat2.sam


Processing SRR978411...


End:21-01-01 14:58:21
Time taken:0:01:04
Start:21-01-01 14:58:21
$ samtools view -@ 6 -o athal_out/SRR978411/SRR978411_hisat2.bam -b athal_out/SRR978411/SRR978411_hisat2.sam
End:21-01-01 14:58:51
Time taken:0:00:30
Start:21-01-01 14:58:51
$ samtools sort -@ 6 -o athal_out/SRR978411/SRR978411_hisat2_sorted.bam athal_out/SRR978411/SRR978411_hisat2.bam
End:21-01-01 14:59:09
Time taken:0:00:18
Start:21-01-01 14:59:10
$ hisat2 --dta-cufflinks -p 6 -x athal_out/athalIndex/athalInd -1 athal_out/SRR971778/SRR971778_1_bbduk.fastq -2 athal_out/SRR971778/SRR971778_2_bbduk.fastq -S athal_out/SRR971778/SRR971778_hisat2.sam


Processing SRR971778...


End:21-01-01 15:00:29
Time taken:0:01:19
Start:21-01-01 15:00:29
$ samtools view -@ 6 -o athal_out/SRR971778/SRR971778_hisat2.bam -b athal_out/SRR971778/SRR971778_hisat2.sam
End:21-01-01 15:01:11
Time taken:0:00:43
Start:21-01-01 15:01:11
$ samtools sort -@ 6 -o athal_out/SRR971778/SRR971778_hisat2_sorted.bam athal_out/SRR971778/SRR971778_hisat2.bam


Alignment done!! Bam files:athal_out/SRR976159/SRR976159_hisat2_sorted.bam,athal_out/SRR978411/SRR978411_hisat2_sorted.bam,athal_out/SRR971778/SRR971778_hisat2_sorted.bam


End:21-01-01 15:01:37
Time taken:0:00:25


## Using samtools
```pyrpipe``` implemnts a basic high-level samtools API through which samtools functionality could be accessed. **Note: that users can also use the python library ```pysam``` to get advance SAM/BAM/VCF/BCF functionality.**

The followin is an example how the pyrpipe tools module can be used

In [8]:
"""
samOb=tools.Samtools()
#sam to sorted bam
bamList=[]
i=0
for sam in samList:
    print("Processing:"+sam)
    thisBam=samOb.sam_sorted_bam(sam,delete_sam=True,delete_bam=True,objectid=sraObjects[i].srr_accession) #add the object id to keep track of process and object. helpful in debugging and reports later
    i+=1
    if thisBam:
        bamList.append(thisBam)
print("Sorted bam files:"+",".join(bamList))

"""

###Some Examples using pysam###
#for details see: https://pysam.readthedocs.io/en/latest/
#import pysam
#pysam.sort("-@","8","-o","sortedBam.bam","in.bam)
#pysam.merge("-@","8","myMerge",*bamList,"-f")

Processing:athal_out/SRR976159/SRR976159_hisat2.sam
$ samtools view -o athal_out/SRR976159/SRR976159_hisat2.bam -@ 6 -b athal_out/SRR976159/SRR976159_hisat2.sam
Time taken:0:00:37
$ samtools sort -o athal_out/SRR976159/SRR976159_hisat2_sorted.bam -@ 6 athal_out/SRR976159/SRR976159_hisat2.bam
Time taken:0:00:18
Processing:athal_out/SRR978411/SRR978411_hisat2.sam
$ samtools view -o athal_out/SRR978411/SRR978411_hisat2.bam -@ 6 -b athal_out/SRR978411/SRR978411_hisat2.sam
Time taken:0:00:29
$ samtools sort -o athal_out/SRR978411/SRR978411_hisat2_sorted.bam -@ 6 athal_out/SRR978411/SRR978411_hisat2.bam
Time taken:0:00:17
Processing:athal_out/SRR971778/SRR971778_hisat2.sam
$ samtools view -o athal_out/SRR971778/SRR971778_hisat2.bam -@ 6 -b athal_out/SRR971778/SRR971778_hisat2.sam
Time taken:0:00:37
$ samtools sort -o athal_out/SRR971778/SRR971778_hisat2_sorted.bam -@ 6 athal_out/SRR971778/SRR971778_hisat2.bam
Time taken:0:00:26
Sorted bam files:athal_out/SRR976159/SRR976159_hisat2_sorted.bam

## Transcript assembly

The `assembly` module in pyrpipe provides classes to perform transcript assembly using Stringtie or Cufflinks these classes extend the `Assembly` class. The `SRA` class implements `assemble()` fuction to easily perform the assembly. The `assemble()` method first check for a `self.bam_path` attribute and if it is present it calls the `perform_assembly` method on the `Assembler` object. After performing the assembly, the resultant GTF files is saved with the SRA object as `gtf` attribute.
In this example, we will use stringtie to perform transcript assembly.

In [35]:
st=assembly.Stringtie(guide=GTF,threads=4)
gtfList=[]
i=0

for ob in sraObjects:
    print("Processing {}...".format(ob.srr_accession))
    ob.assemble(st)
    gtfList.append(ob.gtf)


#another way is to pass bam files to stringtie object
#for bam in bamList:
#    print("Processing:"+bam)
#    gtfList.append(st.perform_assembly(bam,reference_gtf=GTF,objectid=sraObjects[i].srr_accession))
#    i+=1

print("Final GTFs:"+",".join(gtfList))

Processing SRR976159...


Start:21-01-01 15:09:45
$ stringtie -p 4 -G athal_out/Arabidopsis_thaliana.TAIR10.45.gtf -o athal_out/SRR976159/SRR976159_hisat2_sorted_stringtie.gtf athal_out/SRR976159/SRR976159_hisat2_sorted.bam
End:21-01-01 15:10:21
Time taken:0:00:36
Start:21-01-01 15:10:21
$ stringtie -p 4 -G athal_out/Arabidopsis_thaliana.TAIR10.45.gtf -o athal_out/SRR978411/SRR978411_hisat2_sorted_stringtie.gtf athal_out/SRR978411/SRR978411_hisat2_sorted.bam


Processing SRR978411...


End:21-01-01 15:10:52
Time taken:0:00:31
Start:21-01-01 15:10:52
$ stringtie -p 4 -G athal_out/Arabidopsis_thaliana.TAIR10.45.gtf -o athal_out/SRR971778/SRR971778_hisat2_sorted_stringtie.gtf athal_out/SRR971778/SRR971778_hisat2_sorted.bam


Processing SRR971778...
Final GTFs:athal_out/SRR976159/SRR976159_hisat2_sorted_stringtie.gtf,athal_out/SRR978411/SRR978411_hisat2_sorted_stringtie.gtf,athal_out/SRR971778/SRR971778_hisat2_sorted_stringtie.gtf


End:21-01-01 15:11:28
Time taken:0:00:36


## Generating analysis reports

All the commands executed via pyrpipe are extensively logged. These logs can be easily parsed with the `pyrpipe_diagnostic` command.
The pyrpipe_diagnostic utility lets user generate different types of reports and summaries. 

The Following commands can be run from shell.



### Quick summary of the log


In [ ]:
pyrpipe_diagnostic report --summary pyrpipe_logs/2020-03-16-14_33_21_pyrpipe.log

**Generate a pdf report**
[Output](https://github.com/urmi-21/pyrpipe/blob/master/case_studies/Athaliana_transcript_assembly/2020-03-16-14_33_21_pyrpipe.pdf)

In [11]:
!pyrpipe_diagnostic report pyrpipe_logs/2020-03-16-14_33_21_pyrpipe.log

Report written to 2020-03-16-14_33_21_pyrpipe.pdf


***Dump all commands to a shell file***
[Output](https://github.com/urmi-21/pyrpipe/blob/master/case_studies/Athaliana_transcript_assembly/2020-03-16-14_33_21_pyrpipe.sh)

In [12]:
!pyrpipe_diagnostic shell pyrpipe_logs/2020-03-16-14_33_21_pyrpipe.log

Generating bash script
shell commands written to 2020-03-16-14_33_21_pyrpipe.sh


**Generate multiqc report**
[Output](https://github.com/urmi-21/pyrpipe/blob/master/case_studies/Athaliana_transcript_assembly/multiqc_report.html)

In [13]:
!pyrpipe_diagnostic multiqc -r pyrpipe_logs/2020-03-16-14_33_21_pyrpipe.log

Generating html report with multiqc
[INFO   ]         multiqc : This is MultiQC v1.8
[INFO   ]         multiqc : Template    : default
[INFO   ]         multiqc : Searching   : /home/usingh/work/urmi/hoap/pyrpipe/case_studies/Athaliana_transcript_assembly/tmp
[INFO   ]         bowtie2 : Found 3 reports
[INFO   ]         multiqc : Compressing plot data
[INFO   ]         multiqc : Report      : multiqc_report.html
[INFO   ]         multiqc : Data        : multiqc_data
[INFO   ]         multiqc : MultiQC complete
Removing /home/usingh/work/urmi/hoap/pyrpipe/case_studies/Athaliana_transcript_assembly/tmp/SRR976159_fasterq-dump.txt
Removing /home/usingh/work/urmi/hoap/pyrpipe/case_studies/Athaliana_transcript_assembly/tmp/SRR978411_fasterq-dump.txt
Removing /home/usingh/work/urmi/hoap/pyrpipe/case_studies/Athaliana_transcript_assembly/tmp/SRR971778_fasterq-dump.txt
Removing /home/usingh/work/urmi/hoap/pyrpipe/case_studies/Athaliana_transcript_assembly/tmp/SRR976159_bbduk.sh.txt
Removing /ho

**Generate runtime benchmarks**
[Output](https://github.com/urmi-21/pyrpipe/tree/master/case_studies/Athaliana_transcript_assembly/benchmark_reports)

In [14]:
!pyrpipe_diagnostic benchmark pyrpipe_logs/2020-03-16-14_33_21_pyrpipe.log

Generating benchmarks
parsing log...
done.
Benchmark report saved to:/home/usingh/work/urmi/hoap/pyrpipe/case_studies/Athaliana_transcript_assembly/tmp/benchmark_reports
